# Lab 2B – Base 64 y XOR

## Requisito

In [90]:
def decimalToBinary(decimal, bits=8):
    binary = ''
    while decimal > 0:
        binary = str(decimal % 2) + binary
        decimal = decimal // 2
    
    binary = binary.rjust(bits, '0')
    return binary

def ascciToBinary(ascci):
    return ' '.join(decimalToBinary(ord(char)) for char in ascci)

def base64ToAscii(base64):
    index = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    padding = base64.count("=") // 2
    base64 = base64.replace("=", "")
    temp = ''.join(decimalToBinary(index.index(char), bits=6) for char in base64)
    temp = temp[0:len(temp) - padding]
    return ''.join(chr(int(temp[i:i + 8], 2)) for i in range(0, len(temp), 8))

def cleanText(text):
    text = text.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").lower()

    return ''.join(char for char in text if char.isalpha() or char == "_")

def xor(text, key):
    #text = cleanText(text)
    #key = cleanText(key)

    if len(text) > len(key):
        key = key * (len(text) // len(key)) + key[:len(text) % len(key)]
    elif len(text) < len(key):
        key = key[:len(text)]
        
    bin1 = ascciToBinary(text).replace(" ", "")
    bin2 = ascciToBinary(key).replace(" ", "")

    res = ""
    for i in range(len(bin1)):
        if bin1[i] == bin2[i]:
            res += "0"
        else:
            res += "1"
    return res

## Problemas

## Problema 2

Dada la imagen XOR_Imagen, y la llave “cifrados_2025” encontrar el valor original de la imagen.

In [ ]:
# Código terminado con ayuda de ChatGpt4o https://chatgpt.com/share/67ad7fae-50bc-8003-afbb-54ae0d784104
import base64

imageBase64 = None
with open("./images/xor/imagen_xor.png", "rb") as image:
    imageBase64 = base64.b64encode(image.read()).decode()

imageBase64 = base64ToAscii(imageBase64)
key = "cifrados_2025"

xorImageBin = xor(imageBase64, key)
xorImageBytes = bytearray(int(xorImageBin[i:i+8], 2) for i in range(0, len(xorImageBin), 8))

with open("./images/xor/imagen_xor_descifrada.png", "wb") as image:
    image.write(xorImageBytes)

## Problema 3

__Investigar porque al aplicar XOR con una llave de texto la imagén se corrompe__

Esto se debe a que al aplicar un XOR a la imagen, inevitablemente se compromete el header de ésta. La cual contiene información crucial de la imagen. Tal como metadata, información de copyright (realmente no importa), ajustes de compresión, profundidad de color de la imagen, etc. Esas últimas son de suma importancia, esas y otras hacen que la imagen pueda ser visualizada. Por lo que modificar ciertas partes del header de la imagen hacen que ésta sea ilegible por cualquier visualizador de imágenes. Por lo que la imagen se corrompe al modificarle dichas partes.

[Referencia](https://www.quora.com/What-is-the-header-of-an-image)

## Problema 4

Investigar como aplicar un xor a 2 imagnes. Para esto deben de eleccionar 2 imágenes, luego proceder hacer un xor entre las dos imágenes. Esto significa que una imagen es la original y la otra se utilizará como llave para aplicar el xor.

In [16]:
# https://stackoverflow.com/questions/54398627/xor-ing-and-summing-two-black-and-white-images
from PIL import Image
import numpy as np

im1 = Image.open("./images/merge/imagen3.jpg")
widthIm1, heightIm1 = im1.size
im2 = Image.open("./images/merge/imagen4.jpg")
widthIm2, heightIm2 = im2.size

if widthIm1 != widthIm2 or heightIm1 != heightIm2:
    im2 = im2.resize((widthIm1, heightIm1), Image.LANCZOS)

# Make into Numpy arrays
im1np = np.array(im1)*255
im2np = np.array(im2)*255

# XOR with Numpy
result = np.bitwise_xor(im1np, im2np).astype(np.uint8)

# Convert back to PIL image and save
Image.fromarray(result).save('./images/merge/result.jpg')